Notes:
Can't find: interprofessional
Noun: quality
Transformed: surprising, intriguing, and motivated -> searched instead for surprise, intrigue, and motivate
biggest -> find "big" then manually change to superlative
No synonyms: e.g., synergistic, transdisciplinary 

# Conclusion:
Trends paper noted the words that increased in the past 4 decades.
Although many of our synonyms by themselves seem promotional, but maybe their frequencies haven't incrased much

Randomly drew from syns+hype: significantly lower
syns in corpus+hype: significantly lower
syns 1000+ in corpus+hype: significantly lower

In [2]:
import math
import numpy as np
import pandas as pd
import pickle
import re
import string
import random
import logging
import os
import time

import requests
import json

from gensim.models import Word2Vec

from nltk.tokenize import sent_tokenize
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from scipy.stats import chisquare

from sklearn.feature_extraction.text import CountVectorizer as CV

from langdetect import detect
from collections import defaultdict


SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

/Users/huilianq/miniconda3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
# DON'T RUN
app_id = "2d183a33"
app_key = "64e60454062483bfe53b556d2f00407d"
language = "en-us"
word_id = "compelling"
url = "https://od-api.oxforddictionaries.com:443/api/v2/entries/" +\
                language + "/" + word_id.lower()
r = requests.get(url, headers={"app_id": app_id, "app_key": app_key})

In [4]:
hype_dict = {
'Importance': ['compelling',
 'critical',
 'crucial',
 'essential',
 'foundational',
 'fundamental',
 'imperative',
 'important',
 'indispensable',
 'invaluable',
 'key',
 'major',
 'paramount',
 'pivotal',
 'significant',
 'strategic',
 'timely',
 'ultimate',
 'urgent',
 'vital'],

'Novelty': ['creative',
 'emerging',
 'first',
 'groundbreaking',
 'innovative',
 'latest',
 'novel',
 'revolutionary',
 'unique',
 'unparalleled',
 'unprecedented'],
             
'Rigor': ['accurate',
 'advanced',
 'careful',
 'cohesive',
 'detailed',
 'nuanced',
 'powerful',
 'quality',
 'reproducible',
 'rigorous',
 'robust',
 'scientific',
 'sophisticated',
 'strong',
 'systematic'],
             
'Scale': ['ample',
 'biggest',
 'broad',
 'comprehensive',
 'considerable',
 'deeper',
 'diverse',
 'enormous',
 'expansive',
 'extensive',
 'fastest',
 'greatest',
 'huge',
 'immediate',
 'immense',
 'interdisciplinary',
 'international',
 'interprofessional',
 'largest',
 'massive',
 'multidisciplinary',
 'myriad',
 'overwhelming',
 'substantial',
 'top',
 'transdisciplinary',
 'tremendous',
 'vast'],
             
'Utility': ['accessible',
 'actionable',
 'deployable',
 'durable',
 'easy',
 'effective',
 'efficacious',
 'efficient',
 'generalizable',
 'ideal',
 'impactful',
 'intuitive',
 'meaningful',
 'productive',
 'ready',
 'relevant',
 'rich',
 'safer',
 'scalable',
 'seamless',
 'sustainable',
 'synergistic',
 'tailored',
 'tangible',
 'transformative',
 'userfriendly'],
             
'Quality': ['ambitious',
 'collegial',
 'dedicated',
 'exceptional',
 'experienced',
 'intellectual',
 'longstanding',
 'motivated',
 'premier',
 'prestigious',
 'promising',
 'qualified',
 'renowned',
 'senior',
 'skilled',
 'stellar',
 'successful',
 'talented',
 'vibrant'],
             
'Attitude': ['attractive',
 'confident',
 'exciting',
 'incredible',
 'interesting',
 'intriguing',
 'notable',
 'outstanding',
 'remarkable',
 'surprising'],
             
'Problem': ['alarming',
 'daunting',
 'desperate',
 'devastating',
 'dire',
 'dismal',
 'elusive',
 'stark',
 'unanswered',
 'unmet']
}

hype_list = list(set([j for l in hype_dict.values() for j in l]))

In [ ]:
hype_syns = defaultdict(list)
cant_find = []

In [230]:
# DON'T RUN
# get all synonyms

cant_find = []
hype_syns = defaultdict(set)

count = 0
for hype in hype_list:
    count += 1
    if count % 50 == 0:
        time.sleep(60)
    url = "https://od-api.oxforddictionaries.com:443/api/v2/entries/" +\
                language + "/" + hype.lower()
    r = requests.get(url, headers={"app_id": app_id, "app_key": app_key})
#     print(r.text)
    try:
        j = json.loads(r.text)
    except:
        print(r.text)
        cant_find.append(hype)
        continue
    if "results" not in j: 
        cant_find.append(hype)
        continue
    for res in j["results"]:
        for lex in res["lexicalEntries"]:
            print(lex["lexicalCategory"]["id"])
            if lex["lexicalCategory"]["id"] != "adjective": continue
            for entry in lex["entries"]:
                for sense in lex["entries"][0]["senses"]:
                    if "synonyms" in sense:
                        syns = sense["synonyms"]
                        for syn in syns:
                            hype_syns[hype].add(syn["text"])
                    if "subsenses" not in sense: continue
                    for sub in sense["subsenses"]:
                        if "synonyms" not in sub:
                            continue
                        syns = sub["synonyms"]
                        for syn in syns:
                            hype_syns[hype].add(syn["text"])
# print(hype_syns[hype])
# j

adjective
noun
adjective
adjective
noun
adjective
noun
adjective
adjective
noun
verb
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
noun
adjective
adjective
noun
adjective
adjective
noun
adjective
noun
adjective
adjective
noun
adjective
adjective
adjective
adjective
adjective
adjective
adjective
noun
adjective
adjective
adjective
adjective
noun
adjective
adjective
noun
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
adjective
noun
adjective
adjective
noun
verb
adjective
adjective
noun
adjective
adjective
noun
adjective
adjective
noun
adjective
adjective
adjective
noun
adjective
adjective
adverb
interjection
adjective
adjective
noun
adjective
adjective
adjective
adjective
adjective
noun
adjective
noun
adjective
adjective
adjective
noun
adjective
adjective
noun
adjective
adjective
adjective
adjective
adjective
noun
adjective
verb
noun
adjective
adjective


In [507]:
hype_syns
out = open("../results/hype_all_syn_oxford.csv", "w")
out.write("hype,syn\n")
for hype in hype_syns:
    for syn in list(hype_syns[hype]):
        out.write(hype+","+syn+"\n")
out.close()

In [351]:
set(hype_list) - set(hype_syns.keys())

{'biggest',
 'deeper',
 'emerging',
 'fastest',
 'greatest',
 'interprofessional',
 'largest',
 'motivated',
 'safer',
 'userfriendly'}

In [547]:
# hype_syns["biggest"] = hype_syns["big"]
j


{'id': 'actionable',
 'metadata': {'operation': 'retrieve',
  'provider': 'Oxford University Press',
  'schema': 'RetrieveEntry'},
 'results': [{'id': 'actionable',
   'language': 'en-us',
   'lexicalEntries': [{'entries': [{'pronunciations': [{'dialects': ['American English'],
         'phoneticNotation': 'respell',
         'phoneticSpelling': 'ˈakSH(ə)nəb(ə)l'},
        {'audioFile': 'https://audio.oxforddictionaries.com/en/mp3/actionable__us_1.mp3',
         'dialects': ['American English'],
         'phoneticNotation': 'IPA',
         'phoneticSpelling': 'ˈækʃ(ə)nəb(ə)l'}],
       'senses': [{'definitions': ['giving sufficient reason to take legal action'],
         'domainClasses': [{'id': 'law', 'text': 'Law'}],
         'domains': [{'id': 'law', 'text': 'Law'}],
         'examples': [{'text': 'slanderous remarks are actionable'}],
         'id': 'm_en_gbus0007790.006',
         'shortDefinitions': ['giving sufficient reason to take legal action']},
        {'definitions': ['abl

In [504]:
# hype_syns["biggest"] = hype_syns["big"]
# j
hype_syns.pop("")
tailor
emerge
motivate


dict_keys(['creative', 'meaningful', 'fundamental', 'ideal', 'systematic', 'major', 'critical', 'extensive', 'sustainable', 'efficacious', 'rich', 'elusive', 'attractive', 'compelling', 'unanswered', 'accessible', 'vital', 'ultimate', 'interesting', 'latest', 'revolutionary', 'diverse', 'paramount', 'enormous', 'expansive', 'overwhelming', 'stark', 'productive', 'intellectual', 'relevant', 'desperate', 'devastating', 'premier', 'indispensable', 'unique', 'scientific', 'promising', 'outstanding', 'prestigious', 'unparalleled', 'pivotal', 'myriad', 'ready', 'qualified', 'effective', 'exciting', 'notable', 'important', 'senior', 'dismal', 'broad', 'considerable', 'easy', 'tremendous', 'confident', 'talented', 'innovative', 'robust', 'vast', 'incredible', 'novel', 'actionable', 'tangible', 'renowned', 'essential', 'skilled', 'substantial', 'dire', 'key', 'dedicated', 'efficient', 'rigorous', 'experienced', 'timely', 'careful', 'massive', 'unprecedented', 'interdisciplinary', 'huge', 'compr

In [5]:
df = pd.read_parquet("../data/20220602 - Application full text.parquet",
                     engine="fastparquet",
#                      header=0, 
#                      dtype={'Application_id': str,})
#                      "clean_text": str}
                    )
df = df[["Application_id", "Project_description_clean"]]
# df = df.dropna()
# df = df.fillna(0)
# df["text"] = df["clean_text"]
df["text"] = df["Project_description_clean"]
# custom_stopwords = set(stopwords.words("english"))

dat = pd.read_csv("../data/reg_data_R.csv", dtype={'Application_id': str,})
dat_df = df.merge(dat, on="Application_id")

<ipython-input-5-ceba7751dc7c>:14: DtypeWarning: Columns (21,41,45,52) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv("../data/reg_data_R.csv", dtype={'Application_id': str,})


In [629]:
dat_df.columns.tolist()

['Application_id',
 'Project_description_clean',
 'text',
 'DPSource',
 'Application source record id',
 'Application reference',
 'Application reference numeric',
 'Count as application',
 'Call Grant Officer',
 'Application type',
 'Application type source',
 'Application status',
 'Application status source',
 'Grant reporting source',
 'Grant last reported date and time',
 'Project title',
 'Time dimension date',
 'Time dimension date explanation',
 'Application date',
 'Project start date',
 'Project end date',
 'Active grant flag',
 'Foundation id',
 'Applicant type',
 'Grant reference',
 'Grant giving model',
 'Count as grant',
 'Granted date',
 'Granted amount',
 'Applied amount',
 'Project granted amount',
 'Prize granted amount',
 'Indirect granted amount',
 'Call id',
 'Call source record id',
 'Call name',
 'Main applicant id',
 'Main applicant source record id',
 'Main applicant age',
 'Main applicant age group',
 'Main applicant age group sort',
 'Main applicant gender',


In [8]:
len(dat_df)

15942

In [6]:
def clean_text(text, tokenizer):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
#     text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
#     text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
#     text = re.sub(r"\w+|", "", text)  # Remove ellipsis (and last word)
#     text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation
    
#     text = re.sub("\d+", "", text) # remove digits

    tokens = tokenizer(text)  # Get tokens from text
#     tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
#     tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [7]:
dat_df["text"] = dat_df["text"].fillna("")

# dat_df["english"] = dat_df["text"].map(lambda x:detect(x)=="en")

dat_df["tokens"] = dat_df["text"].map(lambda x: clean_text(x, word_tokenize))

In [9]:
# get hype words' synonyms from saved dictionary
hype_file = open("../results/hype_all_syn_oxford.csv")
hype_syns = [t.strip().split(",") for t in hype_file.readlines()[1:]]
hype_all_syns = defaultdict(list)
for hype_syn in hype_syns:
    hype_all_syns[hype_syn[0]].append(hype_syn[1])
for hype in hype_all_syns:
    hype_all_syns[hype].append(hype)
    
# for hype in hype_all_syns:
#     print(list(hype_all_syns[hype]))
#     if len(list(hype_all_syns[hype])) == 0:
#         print(hype)
#         hype_all_syns.pop(hype)
"synergistic" in hype_all_syns
hype_all_syns["biggest"]

['weighty',
 'vital',
 'vastest',
 'tremendous',
 'titanic',
 'thickset',
 'substantial',
 'sturdiest',
 'stupendous',
 'strapping',
 'stocky',
 'solidest',
 'solemnest',
 'sizeable',
 'significant',
 'serious',
 'rugged',
 'robustest',
 'prodigious',
 'mountainous',
 'monumental',
 'monstrous',
 'momentous',
 'mightiest',
 'megalithic',
 'meatiest',
 'massive',
 'mammoth',
 'major',
 'lustiest',
 'largest',
 'key',
 'important',
 'immense',
 'hugest',
 'high-priority',
 'hefty',
 'greatest',
 'gravest',
 'gigantic',
 'giant',
 'gargantuan',
 'far-reaching',
 'extensive',
 'epic',
 'enormous',
 'elephantine',
 'crucial',
 'critical',
 'considerable',
 'consequential',
 'colossal',
 'burliest',
 'bulkiest',
 'brawniest',
 'biggest']

In [10]:
# TODO: find the syns that appear in our corpus
all_syns = set() # used as a vocab for cv
for hype in hype_all_syns:
    all_syns = all_syns.union(hype_all_syns[hype])

cv = CV(decode_error='ignore', 
        min_df=100, 
        max_df=0.7, 
        ngram_range=(1,1),
        binary=False,
#         max_features=20000,
        vocabulary=list(all_syns)
       )
counts_mat = cv.fit_transform(dat_df["text"]).toarray()

/Users/huilianq/miniconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:1330: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [11]:
"welcoming" in cv.vocabulary_
cv.vocabulary_["welcoming"]
# sum(counts_mat[1436,:])

30

In [12]:
hype_in_syns = {}
all_syns = set()
for hype in hype_all_syns:
    cur_syns = list(hype_all_syns[hype])
    cur_syns = [t for t in cur_syns if t in cv.vocabulary_]
    hype_in_syns[hype] = cur_syns
    all_syns = all_syns.union(set(cur_syns))
len(list(all_syns))

1734

In [13]:
for hype in hype_in_syns:
    print(hype, hype_in_syns[hype])

accessible ['welcoming', 'understandable', 'securable', 'realizable', 'ready', 'reachable', 'procurable', 'pleasant', 'penetrable', 'obtainable', 'obliging', 'intelligible', 'informal', 'hospitable', 'graspable', 'gettable', 'friendly', 'fathomable', 'easy-going', 'cordial', 'congenial', 'comprehensible', 'available', 'attainable', 'approachable', 'agreeable', 'affable', 'acquirable', 'achievable', 'accessible', 'accessible']
accurate ['veracious', 'valid', 'unerring', 'unambiguous', 'truthful', 'sure', 'specific', 'right', 'reliable', 'realistic', 'precision', 'precise', 'perfect', 'painstaking', 'minute', 'meticulous', 'literal', 'lifelike', 'lethal', 'faultless', 'faithful', 'fair', 'factual', 'fact-based', 'explicit', 'exact', 'errorless', 'error-free', 'detailed', 'deadly', 'correct', 'close', 'clear-cut', 'careful', 'canonical', 'authoritative', 'authentic', 'accurate']
actionable ['villainous', 'unlawful', 'nefarious', 'lawless', 'lawbreaking', 'illicit', 'illegitimate', 'illega

In [579]:
# ALL SYNANYMS
before_mean = []
after_mean = []
for i in range(10):
    before_dist = []
    after_dist = []
    dat_df[i] = 0
    for ind, row in dat_df.iterrows():
        before_dist.append(sum([1 if t in hype_list else 0 for t in row["tokens"]]))
        # randomly replace hype words
        rand_new_text = [t for t in row["tokens"] if t != "quality"] # remove noun
        
        rand_new_text = [t if t not in hype_all_syns else random.sample(hype_all_syns[t], 1)
                            for t in row["tokens"]]
        # calculate new distribution
        after_dist.append(sum([1 if t in hype_list # or hype_and_syns?
                                      else 0
                                      for t in rand_new_text]))
#     print(before_dist)
#     print(after_dist)
        before_mean.append(np.mean(before_dist))
        after_mean.append(np.mean(after_dist))
        print(np.mean(before_dist), np.mean(after_dist))
    # chisq
#     [c, p] = chisquare(before_dist, after_dist)
#     chi.append(c)
#     pvalue.append(p)

24.0 0.0
23.5 0.0
31.0 0.3333333333333333
30.75 1.0
31.0 0.8
33.5 3.8333333333333335
37.42857142857143 4.285714285714286
38.25 3.875
36.55555555555556 3.5555555555555554
35.6 3.2
36.45454545454545 3.0
35.416666666666664 2.75
35.07692307692308 2.769230769230769
34.785714285714285 2.5714285714285716
34.2 2.466666666666667
35.8125 2.375
34.23529411764706 2.3529411764705883
34.27777777777778 2.6666666666666665
33.8421052631579 2.6315789473684212
33.05 2.5
32.333333333333336 2.4285714285714284
32.77272727272727 2.3181818181818183
32.0 2.217391304347826
32.375 2.3333333333333335
32.2 2.28
32.92307692307692 2.3076923076923075
32.851851851851855 2.259259259259259
33.0 2.5
32.0 2.4827586206896552
31.2 2.433333333333333
30.258064516129032 2.3548387096774195
29.84375 2.28125
30.727272727272727 2.272727272727273
30.705882352941178 2.2058823529411766
30.428571428571427 2.142857142857143
30.75 2.1666666666666665
30.594594594594593 2.108108108108108
30.842105263157894 2.0789473684210527
30.4871794871

25.04230769230769 1.9038461538461537
24.973180076628353 1.896551724137931
24.938931297709924 1.8893129770992367
24.897338403041825 1.9239543726235742
24.890151515151516 1.9204545454545454
24.958490566037735 1.9132075471698113
24.966165413533833 1.906015037593985
24.917602996254683 1.902621722846442
24.854477611940297 1.8955223880597014
24.970260223048328 1.8884758364312269
24.974074074074075 1.8814814814814815
24.95571955719557 1.9261992619926198
24.926470588235293 1.9227941176470589
24.98901098901099 1.9377289377289377
25.098540145985403 1.9306569343065694
25.06909090909091 1.9236363636363636
25.036231884057973 1.9275362318840579
24.9927797833935 1.924187725631769
24.93525179856115 1.920863309352518
24.967741935483872 1.92831541218638
25.05 1.9357142857142857
25.04270462633452 1.9288256227758007
25.0177304964539 1.9219858156028369
25.063604240282686 1.9505300353356891
25.049295774647888 1.9507042253521127
25.003508771929823 1.9508771929824562
25.006993006993007 1.9475524475524475
25.0

23.994 1.842
23.94610778443114 1.8383233532934131
23.970119521912352 1.8346613545816732
23.958250497017893 1.831013916500994
23.918650793650794 1.8273809523809523
23.91089108910891 1.8277227722772278
23.92094861660079 1.8241106719367588
23.899408284023668 1.8205128205128205
23.856299212598426 1.8169291338582678
23.858546168958743 1.8153241650294696
23.856862745098038 1.815686274509804
23.84931506849315 1.812133072407045
23.884765625 1.814453125
23.83820662768031 1.810916179337232
23.85408560311284 1.8093385214007782
23.858252427184468 1.807766990291262
23.843023255813954 1.823643410852713
23.833655705996133 1.8201160541586074
23.787644787644787 1.8166023166023166
23.753371868978807 1.813102119460501
23.75 1.8115384615384615
23.723608445297504 1.8080614203454894
23.726053639846743 1.8045977011494252
23.720841300191204 1.8011472275334608
23.717557251908396 1.797709923664122
23.742857142857144 1.7980952380952382
23.724334600760457 1.7946768060836502
23.734345351043643 1.7950664136622392
2

23.61917808219178 1.621917808219178
23.62517099863201 1.6265389876880985
23.651639344262296 1.6270491803278688
23.63710777626194 1.6248294679399726
23.6566757493188 1.6226158038147138
23.635374149659864 1.620408163265306
23.61820652173913 1.6182065217391304
23.58751696065129 1.616010854816825
23.558265582655828 1.613821138211382
23.56292286874154 1.6143437077131257
23.541891891891893 1.6135135135135135
23.510121457489877 1.611336032388664
23.52021563342318 1.6118598382749325
23.516823687752357 1.6123822341857335
23.530913978494624 1.6129032258064515
23.514093959731543 1.610738255033557
23.51608579088472 1.6085790884718498
23.495314591700133 1.606425702811245
23.482620320855617 1.6042780748663101
23.468624833110816 1.6021361815754338
23.458666666666666 1.6026666666666667
23.44607190412783 1.6018641810918774
23.470744680851062 1.601063829787234
23.45551128818061 1.598937583001328
23.456233421750664 1.596816976127321
23.449006622516556 1.594701986754967
23.465608465608465 1.59259259259259

21.03441084462982 1.3785192909280501
21.054166666666667 1.384375
21.032258064516128 1.3829344432882413
21.011434511434512 1.3814968814968815
21.008307372793354 1.3811007268951194
21.028008298755186 1.3817427385892116
21.008290155440413 1.3803108808290157
20.989648033126294 1.3788819875776397
20.96794208893485 1.377456049638056
20.950413223140497 1.3760330578512396
20.959752321981423 1.3746130030959753
20.938144329896907 1.3731958762886598
20.916580844490216 1.37178166838311
20.895061728395063 1.3703703703703705
20.875642343268243 1.36896197327852
20.90759753593429 1.3788501026694044
20.886153846153846 1.3774358974358973
20.866803278688526 1.3760245901639345
20.902763561924257 1.374616171954964
20.88445807770961 1.3732106339468302
20.86414708886619 1.3718079673135852
20.88061224489796 1.376530612244898
20.859327217125383 1.3751274209989806
20.838085539714868 1.3737270875763747
20.816887080366225 1.3723296032553407
20.796747967479675 1.3709349593495934
20.80710659898477 1.370558375634517

...

27.960321916526297 1.4858693617817706
27.959019461077844 1.4855913173652695
27.95977549111319 1.485313376987839
27.96090534979424 1.4852225963337073
27.958107349915842 1.4851318496353094
27.957180254300674 1.4850411368735976
27.959244718639 1.4849504580295383
27.958878504672896 1.485607476635514
27.957391141842646 1.485703606802467
27.959267563527654 1.4859865470852018
27.958901550532413 1.485708948253316
27.956667911841613 1.4854314531191632
27.95406162464986 1.48515406162465
27.952203136669155 1.4850634802091114
27.95221205898824 1.484786260966959
27.953154162000747 1.484695782008212
27.950737077813024 1.4846053368165704
27.94776119402985 1.4845149253731342
27.946838276440964 1.4842380152956538
27.94684819097352 1.4839612085042895
27.946112250606003 1.4838709677419355
27.945563012677106 1.483594332587621
27.941845293569433 1.4833178005591798
27.938501677226984 1.483227730152814
27.937581516675984 1.4829513694801566
27.93908345752608 1.4830476900149032
27.93890854907804 1.482771465822

28.07576571735626 1.4914920293748881
28.07593123209169 1.491583094555874
28.074664279319606 1.4913160250671442
28.07017543859649 1.4910490511994272
28.074816538392696 1.4913191337032397
28.074266284896208 1.4912312097351468
28.074610842726784 1.490964394346037
28.074955277280857 1.4908765652951699
28.07512072974423 1.490609908781971
28.074213161659515 1.4910586552217453
28.07652422671196 1.4911496513499016
28.079728280300323 1.4915981408652128
28.082931188561215 1.491331546023235
28.08649035025018 1.491422444603288
28.08468822583527 1.491155976415937
28.090032154340836 1.4910682386566632
28.089301661010897 1.4908019289158778
28.09 1.4907142857142857
28.089091233708267 1.490448134261739
28.087468761156728 1.4901820778293466
28.089416384079957 1.4908084954488667
28.092077087794433 1.490899357601713
28.09491525423729 1.4908117752007137
28.094363182304672 1.4907242240456653
28.097913322632422 1.4906367041198503
28.099679029957205 1.4907275320970044
28.102513817079693 1.4908183276876448
28.

28.41003608867503 1.5059288537549407
28.40824742268041 1.5058419243986254
28.414705377082974 1.5055832331214567
28.420989350738576 1.5060116798351082
28.42314957925468 1.506096513824489
28.418440934065934 1.5058379120879122
28.41854077253219 1.5055793991416309
28.418640576725025 1.50652248541023
28.42491848292432 1.506778788398833
28.42604667124228 1.5068634179821552
28.423571796191457 1.5066049065019729
28.421612349914238 1.506861063464837
28.42582747384668 1.5071171325673127
28.42764060356653 1.5068587105624143
28.43133893365335 1.5069432539002228
28.435550222831676 1.5066849502913953
28.43256212510711 1.506426735218509
28.436771761480465 1.5061686086360522
28.437039575124206 1.5069384958026384
28.437307297019526 1.5066803699897224
28.438945024833018 1.5076211680082205
28.443150684931506 1.5080479452054794
28.441533983906865 1.5079609655880841
28.440089010612805 1.5077028414926394
28.440698271435906 1.5074448057504706
28.442676249144423 1.5075290896646132
28.44824636441403 1.50932420

28.618725231175695 1.517007926023778
28.624731715370647 1.5169225689285124
28.622317596566525 1.5174975239352921
28.623205149364583 1.5172470704736756
28.622442244224423 1.517161716171617
28.62382445141066 1.5170763900346478
28.622896733751237 1.5168261299901022
28.623288800923635 1.516740887349497
28.63126649076517 1.5176451187335092
28.633470733718056 1.5173948887056883
28.630893504780744 1.5171447411803496
28.632932256469424 1.5170595022251525
28.631674357284112 1.5168094924192486
28.633877080243863 1.5165595650024717
28.632454695222407 1.5163097199341022
28.63004447372756 1.5160599571734474
28.62895256916996 1.516304347826087
28.6323069323234 1.518524617157912
28.63302601251235 1.518274613105038
28.63111111111111 1.5181893004115226
28.630184331797235 1.5179394338380514
28.628599637979267 1.5176896494981076
28.62981243830207 1.5182625863770978
28.633985852936338 1.5181773317979932
28.629276315789475 1.5179276315789474
28.628021706956094 1.5176780134846242
28.62857612627425 1.5174284

KeyboardInterrupt: 

In [ ]:
# only the synonyms that appeared in our corpus

results = {}
for ind, row in dat_df.iterrows():
    results[row["Application_id"]] = []

hype_set = set(hype_list)

for i in range(100):
    print(i)
    for ind, row in dat_df.iterrows():
        before_dist.append(sum([1 if t in hype_list else 0 for t in row["tokens"]]))
        # randomly replace hype words
        rand_new_text = [t for t in row["tokens"] if t != "quality"] # remove noun

#         num_hype = 0
#         for tind, t in enumerate(rand_new_text):
#             if t in hype_in_syns.keys():
#                 new_word = random.sample(set(hype_in_syns[t]), 1)[0]
#                 rand_new_text[tind] = new_word
#                 if new_word in hype_set:
# #                     print(t, new_word)
#                     num_hype += 1
#                 print(t, rand_new_text[tind])
        rand_new_text = [t if t not in hype_in_syns.keys() else random.sample(set(hype_in_syns[t]), 1)[0]
                            for t in rand_new_text]
    
        results[row["Application_id"]].append(#num_hype)
            sum([1 if t in hype_list 
                                      else 0
                                      for t in rand_new_text]))
#         if ind%1000==0:
#             print(results[row["Application_id"]], dat_df.iloc[ind]["total_num_hype_words"])
#     print(np.mean(before_dist), np.mean(after_dist))

0


<ipython-input-15-e480f767852a>:25: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  rand_new_text = [t if t not in hype_in_syns.keys() else random.sample(set(hype_in_syns[t]), 1)[0]


1


In [664]:
results["NORMA APP id: 1181779"]
dat_df.loc[dat_df["Application_id"]=="NORMA APP id: 1181779"]["total_num_hype_words"]
# list(results.keys())[:6]
results

{'NORMA APP id: 1181768': [2,
  5,
  8,
  8,
  4,
  6,
  2,
  4,
  6,
  6,
  2,
  1,
  5,
  6,
  2,
  2,
  2,
  3,
  4,
  3,
  2,
  7,
  5,
  4,
  1,
  2,
  4,
  2,
  4,
  6,
  3,
  3,
  2,
  3,
  6,
  5,
  7,
  2,
  2,
  4,
  5,
  8,
  4,
  4,
  3,
  4,
  4,
  4,
  2,
  4,
  6,
  3,
  4,
  3,
  5,
  0,
  4,
  5,
  2,
  4,
  4,
  2,
  4,
  5,
  3,
  5,
  5,
  3,
  5,
  2,
  1,
  3,
  5,
  5,
  1,
  1,
  5,
  4,
  3,
  2,
  5,
  4,
  3,
  2,
  4,
  2,
  4,
  6,
  3,
  4,
  3,
  1,
  3,
  8,
  3,
  3,
  7,
  3,
  6,
  4],
 'NORMA APP id: 1181773': [5,
  3,
  3,
  5,
  6,
  5,
  5,
  6,
  10,
  8,
  8,
  6,
  7,
  7,
  3,
  6,
  8,
  6,
  4,
  3,
  6,
  5,
  6,
  4,
  8,
  6,
  8,
  8,
  4,
  8,
  7,
  4,
  5,
  7,
  5,
  5,
  3,
  3,
  5,
  4,
  6,
  7,
  7,
  7,
  8,
  8,
  8,
  7,
  4,
  8,
  4,
  7,
  5,
  11,
  5,
  6,
  5,
  1,
  8,
  4,
  8,
  7,
  5,
  7,
  4,
  7,
  9,
  7,
  8,
  2,
  7,
  8,
  7,
  2,
  3,
  7,
  7,
  8,
  6,
  7,
  5,
  4,
  8,
  6,
  7,
  9,
  6,
  6,
  9,
  

In [675]:
results_df = pd.DataFrame(results)#, columns=range(1,101))# index=results.keys())#, columns=range(1,101))

In [679]:
results_df.transpose().to_csv("random_replacement_in_corpus_syns.csv")